#ReAct Agent with Google GenAI
Made by: Wilfredo Aaron Sosa Ramos (AI Lab Manager at RealityAI Labs)

##1. Install the dependencies

In [1]:
!pip install -q langchain langchain_core langchain_community langchain_google_genai langgraph

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 17.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 25.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 411.6/411.6 kB 19.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.3/41.3 kB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.2/137.2 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.9/48.9 kB 2.4 MB/s eta 0:00:00


##2. Set the env. variables

In [2]:
import os
from google.colab import userdata

os.environ['GOOGLE_API_KEY'] = userdata.get('GOOGLE_API_KEY')

##3. Basic Implementation of a Tool

In [4]:
from langchain_core.tools import tool
from langchain_google_genai import ChatGoogleGenerativeAI

# Initialize the model
model = ChatGoogleGenerativeAI(model="gemini-2.0-flash-exp")

# Define a tool for Project KPI Calculation
@tool
def calculate_kpi(progress: float, planned_effort: int, actual_effort: int, deadline_met: bool) -> dict:
    """
    Calculates KPIs for a project task based on progress, effort, and deadline adherence.

    KPIs include:
    - Efficiency: Ratio of planned to actual effort (percentage).
    - Progress Score: Normalized task progress (0-100 scale).
    - Deadline Compliance: Boolean converted to a score (0 or 100).
    - Overall KPI: Weighted average of all metrics.

    Args:
        progress (float): Task progress as a percentage (0-100).
        planned_effort (int): Planned effort in hours.
        actual_effort (int): Actual effort in hours.
        deadline_met (bool): Whether the deadline was met.

    Returns:
        dict: Dictionary containing calculated KPIs.
    """
    # Calculate individual metrics
    efficiency = min((planned_effort / actual_effort) * 100, 100) if actual_effort > 0 else 0
    progress_score = max(min(progress, 100), 0)  # Ensure within 0-100 range
    deadline_score = 100 if deadline_met else 0

    # Calculate overall KPI (weighted average)
    overall_kpi = round((0.4 * efficiency) + (0.4 * progress_score) + (0.2 * deadline_score), 2)

    # Return KPIs as a dictionary
    return {
        "Efficiency (%)": round(efficiency, 2),
        "Progress Score (%)": progress_score,
        "Deadline Compliance (%)": deadline_score,
        "Overall KPI (%)": overall_kpi,
    }

# List of tools
tools = [calculate_kpi]

# Example usage
progress = 75.0  # 75% task completed
planned_effort = 40  # Planned effort was 40 hours
actual_effort = 50  # Actual effort turned out to be 50 hours
deadline_met = True  # Deadline was met

kpi_results = calculate_kpi.run({
    "progress": progress,
    "planned_effort": planned_effort,
    "actual_effort": actual_effort,
    "deadline_met": deadline_met,
})

# Print KPIs
query = f"The calculated KPIs are: {kpi_results}"
print(query)

The calculated KPIs are: {'Efficiency (%)': 80.0, 'Progress Score (%)': 75.0, 'Deadline Compliance (%)': 100, 'Overall KPI (%)': 82.0}


##4. Agent Executor:

In [6]:
from langchain.agents import AgentExecutor, create_tool_calling_agent
from langchain_core.prompts import ChatPromptTemplate
# Define the agent's prompt
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful assistant capable of calculating project KPIs."),
        ("human", "{input}"),
        # Placeholder for agent's scratchpad to allow dynamic reasoning
        ("placeholder", "{agent_scratchpad}"),
    ]
)

# Create the agent using the tools and prompt
agent = create_tool_calling_agent(model, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools)

# Example query to calculate KPIs
query = (
    "Calculate the KPIs for a task with 75% progress, 40 planned hours, 50 actual hours, "
    "and the deadline was met. Use the tool to provide detailed results."
)

# Execute the agent
result = agent_executor.invoke({"input": query})

# Print the output
print(result)

{'input': 'Calculate the KPIs for a task with 75% progress, 40 planned hours, 50 actual hours, and the deadline was met. Use the tool to provide detailed results.', 'output': 'The KPIs for the task are as follows: Efficiency: 80%, Progress Score: 75%, Deadline Compliance: 100%, and Overall KPI: 82%.\n'}


##5. ReAct Agent Executor:

In [7]:
from langgraph.prebuilt import create_react_agent

langgraph_agent_executor = create_react_agent(model, tools)

messages = langgraph_agent_executor.invoke({"messages": [("human", query)]})
{
    "input": query,
    "output": messages["messages"][-1].content,
}

{'input': 'Calculate the KPIs for a task with 75% progress, 40 planned hours, 50 actual hours, and the deadline was met. Use the tool to provide detailed results.',
 'output': 'The KPIs for the task are as follows: Efficiency: 80%, Progress Score: 75%, Deadline Compliance: 100%, and Overall KPI: 82%.\n'}

##6. ReAct Agent with Prompts:

In [8]:
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful assistant capable of calculating project KPIs. Respond only in Spanish."),
        ("human", "{input}"),
        # Placeholders fill up a **list** of messages
        ("placeholder", "{agent_scratchpad}"),
    ]
)

agent = create_tool_calling_agent(model, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools)

agent_executor.invoke({"input": query})

{'input': 'Calculate the KPIs for a task with 75% progress, 40 planned hours, 50 actual hours, and the deadline was met. Use the tool to provide detailed results.',
 'output': 'Los KPIs para la tarea son: Cumplimiento del plazo: 100%, Eficiencia: 80%, Puntaje de progreso: 75%, KPI general: 82%.\n'}

In [9]:
from langchain_core.messages import SystemMessage
from langgraph.prebuilt import create_react_agent

# Define a professional system prompt
system_message = SystemMessage(content=(
    "You are a highly professional project management assistant specializing in Key Performance Indicators (KPIs). "
    "Your role is to assist users in evaluating task performance using metrics like efficiency, progress, and deadline compliance. "
    "Respond only in Spanish, ensuring your answers are concise, accurate, and professional."
))

# Create the agent executor
langgraph_agent_executor = create_react_agent(
    model, tools, state_modifier=system_message
)

# Invoke the agent with a query
messages = langgraph_agent_executor.invoke({"messages": [("user", query)]})


In [10]:
messages

{'messages': [HumanMessage(content='Calculate the KPIs for a task with 75% progress, 40 planned hours, 50 actual hours, and the deadline was met. Use the tool to provide detailed results.', additional_kwargs={}, response_metadata={}, id='8ec5629a-6f1d-4746-8912-4e5fb235e4c0'),
  AIMessage(content='', additional_kwargs={'function_call': {'name': 'calculate_kpi', 'arguments': '{"progress": 75.0, "deadline_met": true, "planned_effort": 40.0, "actual_effort": 50.0}'}}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': [{'category': 'HARM_CATEGORY_HATE_SPEECH', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_DANGEROUS_CONTENT', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_HARASSMENT', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_SEXUALLY_EXPLICIT', 'probability': 'NEGLIGIBLE', 'blocked': False}]}, id='run-d00bf3ea-f199-451e-9c52-

In [11]:
from langgraph.prebuilt import create_react_agent
from langgraph.prebuilt.chat_agent_executor import AgentState
from langchain_core.prompts import ChatPromptTemplate

# Define a professional system prompt for KPI evaluation
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", (
            "You are a highly professional project management assistant. Your primary responsibility is to calculate "
            "Key Performance Indicators (KPIs) such as efficiency, progress, and deadline compliance. Respond only in Spanish."
        )),
        ("placeholder", "{messages}"),
    ]
)

# State modification to adjust responses and add a specific message
def _modify_state_messages(state: AgentState):
    return prompt.invoke({"messages": state["messages"]}).to_messages() + [
        ("user", "Además, agrega '¡Gracias por Preguntar!' después de la respuesta.")
    ]

# Create the agent executor
langgraph_agent_executor = create_react_agent(
    model, tools, state_modifier=_modify_state_messages
)

# Example realistic query for KPI calculation
query = (
    "Calcule los KPIs para una tarea con 85% de progreso, 40 horas planeadas, "
    "50 horas reales y se cumplió con el plazo. Proporcione los resultados detallados."
)

# Invoke the agent with the query
messages = langgraph_agent_executor.invoke({"messages": [("human", query)]})

# Print the input and the formatted output
print(
    {
        "input": query,
        "output": messages["messages"][-1].content,
    }
)


{'input': 'Calcule los KPIs para una tarea con 85% de progreso, 40 horas planeadas, 50 horas reales y se cumplió con el plazo. Proporcione los resultados detallados.', 'output': 'Los KPIs para la tarea son: Cumplimiento de Plazo: 100%, Eficiencia: 80%, Progreso: 85% y el KPI general es 86%. ¡Gracias por Preguntar!\n'}


In [13]:
print(messages["messages"][-1].content)

Los KPIs para la tarea son: Cumplimiento de Plazo: 100%, Eficiencia: 80%, Progreso: 85% y el KPI general es 86%. ¡Gracias por Preguntar!

